# SIIP PCM Demo

### Clayton Barrows

#### Aug 3, 2022

##### Team:
 - Sourabh Dalvi
 - Dheepak Krishnamurthy
 - Jose Daniel Lara
 - Dan Thom

<img src="./images/siip-power-logo.png" alt="siip::power" style="width:150%;"/>

## The contribution of __MATPOWER__

```matlab
matlab:1> runpf

MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.02 seconds
================================================================================
|     System Summary                                                           |
================================================================================

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers       0     Branch Charging (inj)     -              140.5
Inter-ties         0     Total Inter-tie Flow     0.0               0.0
Areas              1

                          Minimum                      Maximum
                 -------------------------  --------------------------------
Voltage Magnitude   0.996 p.u. @ bus 9          1.040 p.u. @ bus 1   
Voltage Angle      -3.99 deg   @ bus 9          9.28 deg   @ bus 2   
P Losses (I^2*R)             -                  2.30 MW    @ line 8-9
Q Losses (I^2*X)             -                 15.83 MVAr  @ line 8-2
```

### Environment

In [83]:
] st

      Status `~/Documents/repos/PCM-Demo-Presentation/Project.toml`
  [87dc4568] HiGHS v1.1.4
  [f0f68f2c] PlotlyJS v0.18.8
  [5f7eddb3] PowerGraphics v0.12.0
  [e690365d] PowerSimulations v0.15.4
  [f00506e0] PowerSystemCaseBuilder v0.5.1
  [bcd98974] PowerSystems v1.22.4
  [56ddb016] Logging


In [84]:
using PowerSystems
using PowerSimulations
using PowerGraphics
plotlyjs()
using PowerSystemCaseBuilder
using Dates
using Logging;
logger = configure_logging(
    console_level = Logging.Error)
using Xpress
solver = optimizer_with_attributes(
    Xpress.Optimizer, 
    "MIPRELSTOP" => 0.5, "OUTPUTLOG" => 1);

# PowerSystems.jl

## PowerSystems.jl supports parsers for a few standard power system data formats:
 - MATPOWER
 - PSS(R)E v33 .raw *and .dyr* format
 - Tabular data (CSV)

In [85]:
sys_RT = build_system(PSITestSystems, "modified_RTS_GMLC_RT_sys");
sys = build_system(PSITestSystems, "modified_RTS_GMLC_DA_sys")

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ SYSTEM_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 500         │
└───────────────────┴─────────────┘

Static Components
┌──────────────────────────────┬───────┬────────────────────────┬───────────────┐
│ Type                         │ Count │ Has Static Time Series │ Has Forecasts │
├──────────────────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc                          │ 109   │ false                  │ false         │
│ Area                         │ 3     │ true                   │ true          │
│ Bus                          │ 73    │ false                  │ false         │
│ FixedAdmittance              │ 3     │ true                   │ true          │
│ HydroDispatch                │ 1     │ true                   │ true          │
│ Line                         │ 105   │ false                  │ false         │
│ LoadZone                     │ 21    │ false                  │ false         │
│ PowerLoad                    │ 51    │ true                   │ true          │
│ RenewableDispatch            │ 29    │ true                   │ true          │
│ RenewableFix                 │ 31    │ true                   │ true          │
│ TapTransformer               │ 15    │ false                  │ false         │
│ ThermalStandard              │ 54    │ false                  │ false         │
│ VariableReserve{ReserveDown} │ 1     │ true                   │ true          │
│ VariableReserve{ReserveUp}   │ 4     │ true                   │ true          │
└──────────────────────────────┴───────┴────────────────────────┴───────────────┘

Time Series Summary
┌──────────────────────────────────┬─────────────────────┐
│ Property                         │ Value               │
├──────────────────────────────────┼─────────────────────┤
│ Components with time series data │ 123                 │
│ Total StaticTimeSeries           │ 124                 │
│ Total Forecasts                  │ 124                 │
│ Resolution                       │ 60 minutes          │
│ First initial time               │ 2020-01-01T00:00:00 │
│ Last initial time                │ 2020-12-30T00:00:00 │
│ Horizon                          │ 48                  │
│ Interval                         │ 1440 minutes        │
│ Forecast window count            │ 365                 │
└──────────────────────────────────┴─────────────────────┘

### `TimeSeriesData`

 - Any `Device` or `Service` can contain `TimeSeriesData`
 - `TimeSeriesData` data can describe any field in the `Component`, or can be arbitrary values
 - `TimeSeriesData` can represent forecast data or values with a unique set of timestamps 

In [61]:
subtypes(TimeSeriesData)

2-element Vector{Any}:
 Forecast
 StaticTimeSeries

<img src="./images/prefetching.png" alt="siip vision" style="width:100%;"/>

# PowerSimulations.jl

## PowerSimulations.jl supports a variety of optimization problems: 
 - Unit Commitment (UC)
 - Economic Dispatch (ED)
 - Optimal Power Flow (ACOPF, DCOPF)
 - Area Control Error (ACE) & Automatic Generation Control (AGC)



 

 ##### A `Simulation` can be defined to enable a sequence of `DecisionModel`s with control over:

- Problem specification
     - Formulation
     - Size
 - Problem sequencing
 - Information flow between problems
     - Initial conditions
     - Parameters

### Define the template for the day-ahead and real-time problems

In [86]:
template_uc = template_unit_commitment()

Network Model
┌───────────────┬───────────────────────┐
│ Network Model │ CopperPlatePowerModel │
│ Slacks        │ false                 │
│ PTDF          │ false                 │
│ Duals         │                       │
└───────────────┴───────────────────────┘

Device Models
┌──────────────────────┬────────────────────────────┬────────┐
│ Device Type          │ Formulation                │ Slacks │
├──────────────────────┼────────────────────────────┼────────┤
│ ThermalStandard      │ ThermalBasicUnitCommitment │ false  │
│ HydroDispatch        │ HydroDispatchRunOfRiver    │ false  │
│ PowerLoad            │ StaticPowerLoad            │ false  │
│ RenewableFix         │ FixedOutput                │ false  │
│ RenewableDispatch    │ RenewableFullDispatch      │ false  │
│ HydroEnergyReservoir │ HydroDispatchRunOfRiver    │ false  │
│ InterruptibleLoad    │ InterruptiblePowerLoad     │ false  │
└──────────────────────┴────────────────────────────┴────────┘

Branch Models
┌────────────────┬──────────────┬────────┐
│ Branch Type    │ Formulation  │ Slacks │
├────────────────┼──────────────┼────────┤
│ Line           │ StaticBranch │ false  │
│ TapTransformer │ StaticBranch │ false  │
│ Transformer2W  │ StaticBranch │ false  │
│ HVDCLine       │ HVDCDispatch │ false  │
└────────────────┴──────────────┴────────┘

Service Models
┌──────────────────────────────┬──────────────┬────────┬──────────────────┐
│ Service Type                 │ Formulation  │ Slacks │ Aggregated Model │
├──────────────────────────────┼──────────────┼────────┼──────────────────┤
│ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ true             │
│ VariableReserve{ReserveDown} │ RangeReserve │ false  │ true             │
└──────────────────────────────┴──────────────┴────────┴──────────────────┘

In [87]:
template_ed = template_economic_dispatch()

Network Model
┌───────────────┬───────────────────────┐
│ Network Model │ CopperPlatePowerModel │
│ Slacks        │ false                 │
│ PTDF          │ false                 │
│ Duals         │                       │
└───────────────┴───────────────────────┘

Device Models
┌──────────────────────┬─────────────────────────┬────────┐
│ Device Type          │ Formulation             │ Slacks │
├──────────────────────┼─────────────────────────┼────────┤
│ ThermalStandard      │ ThermalBasicDispatch    │ false  │
│ HydroDispatch        │ HydroDispatchRunOfRiver │ false  │
│ PowerLoad            │ StaticPowerLoad         │ false  │
│ RenewableFix         │ FixedOutput             │ false  │
│ RenewableDispatch    │ RenewableFullDispatch   │ false  │
│ HydroEnergyReservoir │ HydroDispatchRunOfRiver │ false  │
│ InterruptibleLoad    │ InterruptiblePowerLoad  │ false  │
└──────────────────────┴─────────────────────────┴────────┘

Branch Models
┌────────────────┬──────────────┬────────┐
│ Branch Type    │ Formulation  │ Slacks │
├────────────────┼──────────────┼────────┤
│ Line           │ StaticBranch │ false  │
│ TapTransformer │ StaticBranch │ false  │
│ Transformer2W  │ StaticBranch │ false  │
│ HVDCLine       │ HVDCDispatch │ false  │
└────────────────┴──────────────┴────────┘

Service Models
┌──────────────────────────────┬──────────────┬────────┬──────────────────┐
│ Service Type                 │ Formulation  │ Slacks │ Aggregated Model │
├──────────────────────────────┼──────────────┼────────┼──────────────────┤
│ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ true             │
│ VariableReserve{ReserveDown} │ RangeReserve │ false  │ true             │
└──────────────────────────────┴──────────────┴────────┴──────────────────┘

### Define the Models


In this case, we want to define two models with the templates and the Systems that we've already created.

 - Each model defines a JuMP model
 - The JuMP model gets updated to represent different time periods

In [88]:
models = SimulationModels(
    decision_models = [
        DecisionModel(template_uc, sys, optimizer = solver, name = "UC", optimizer_solve_log_print = true),
        DecisionModel(template_ed, sys_RT, optimizer = solver, name = "ED", optimizer_solve_log_print = true),
    ],
)

Model Name,Model Type,Status,Output Directory
UC,GenericOpProblem,EMPTY,nothing
ED,GenericOpProblem,EMPTY,nothing


### `SimulationSequence` provides a template of how to execute a sequential set of operations problems

 - FeedForward: defines what information is passed
 - Chronologies: define where information is flowing

 
 <img src="./images/da-rt-agc.jpg" alt="siip vision" style="width:75%;" align="left"/>
 

In [89]:
feedforward = Dict(
    "ED" => [
        SemiContinuousFeedforward(
            component_type = ThermalStandard,
            source = OnVariable,
            affected_values = [ActivePowerVariable],
        ),
    ],
)

Dict{String, Vector{SemiContinuousFeedforward}} with 1 entry:
  "ED" => [SemiContinuousFeedforward(VariableKey{OnVariable, ThermalStandard}(""), PowerSimulations.VariableKey[VariableKey{ActivePowerVariable, ThermalStandard}("")])]

In [90]:
DA_RT_sequence = SimulationSequence(
    models = models,
    ini_cond_chronology = InterProblemChronology(),
    feedforwards = feedforward,
)

Simulation Sequence
┌──────────────────────────┬──────────┐
│ Simulation Step Interval │ 24 hours │
│ Number of Problems       │ 2        │
└──────────────────────────┴──────────┘

Simulation Problems
┌────────────┬─────────┬──────────────┬─────────────────────┐
│ Model Name │ Horizon │ Interval     │ Executions Per Step │
├────────────┼─────────┼──────────────┼─────────────────────┤
│ UC         │ 48      │ 1440 minutes │ 1                   │
│ ED         │ 12      │ 15 minutes   │ 96                  │
└────────────┴─────────┴──────────────┴─────────────────────┘

Feedforwards
┌────────────┬───────────────────────────┐
│ Model Name │ Feed Forward Type         │
├────────────┼───────────────────────────┤
│ ED         │ SemiContinuousFeedforward │
└────────────┴───────────────────────────┘

### Build a `Simulation` using the defined `SimulationSequence`

In [91]:
sim = Simulation(
    name = "rts-test",
    steps = 2,
    models = models,
    sequence = DA_RT_sequence,
    simulation_folder = mktempdir(".", cleanup = true),
)

build!(sim, console_level = Logging.Info)

[ Info: Building problem UC
[ Info: Initializing Optimization Container For a DecisionModel
[ Info: Initializing ModelStoreParams
[ Info: Mapping Service Models
[ Info: Make Initial Conditions Model
[ Info: No initial conditions in the model
[ Info: The data doesn't include devices of type HydroEnergyReservoir, consider changing the device models
[ Info: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
[ Info: The data doesn't include devices of type Transformer2W, consider changing the device models
[ Info: The data doesn't include devices of type HVDCLine, consider changing the device models
[ Info: The data doesn't include devices of type HydroEnergyReservoir, consider changing the device models
[ Info: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
[ Info: The data doesn't include devices of type Transformer2W, consider changing the device models
[ Info: The data doesn't include devices 

BuildStatus.BUILT = 0

### Execute simulation

In [92]:
sim_results = execute!(sim, enable_progress_bar = true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Progress: 100%|█████████████████████████████████████████| Time: 0:00:28
  Step:                  2
  Problem:               ED
  Simulation Timestamp:  2020-01-02T23:45:00


FICO Xpress v8.8.0, Hyper, solve started 11:31:53, Aug 3, 2022
Heap usage: 4191KB (peak 7444KB, 2048KB system)
Minimizing LP 
Original problem has:
      3048 rows         5436 cols        12192 elements
Presolved problem has:
      1476 rows         3288 cols        10128 elements
Presolve finished in 0 seconds
Heap usage: 4844KB (peak 7444KB, 2048KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 2.20e-01,  4.00e+00] / [ 1.10e-01,  1.77e+00]
  RHS and bounds [min,max] : [ 6.94e-18,  3.26e+01] / [ 2.20e-01,  1.63e+01]
  Objective      [min,max] : [ 1.00e-02,  8.29e+02] / [ 1.00e-02,  8.29e+02]

Starting parallel dual simplex, using up to 8 threads
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0       63934.91572      D      3     0        .000000     0
     3       63934.91572      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  3 simplex 

┌ Info: 
│  ────────────────────────────────────────────────────────────────────────────────────
│                                             Time                    Allocations      
│                                    ───────────────────────   ────────────────────────
│          Tot / % measured:              28.2s / 100.0%           6.70GiB / 100.0%    
│ 
│  Section                   ncalls     time    %tot     avg     alloc    %tot      avg
│  ────────────────────────────────────────────────────────────────────────────────────
│  Execute Simulation             1    28.2s  100.0%   28.2s   6.70GiB  100.0%  6.70GiB
│    Execute ED                 192    25.0s   88.7%   130ms   6.25GiB   93.2%  33.3MiB
│      Solve ED                 192    13.3s   47.2%  69.3ms   3.23GiB   48.2%  17.2MiB
│      Update State             192    2.85s   10.1%  14.9ms    263MiB    3.8%  1.37MiB
│      Update ED                192    1.91s    6.8%  10.0ms    391MiB    5.7%  2.04MiB
│        Parameter U

RunStatus.SUCCESSFUL = 0

## Analysis
PowerSimulations.jl natively populates simulation results in a struct of DataFrames. 


In [93]:
results = SimulationResults(sim);
uc_results = get_decision_problem_results(results, "UC");
ed_results = get_decision_problem_results(results, "ED")

EnergyOutput__HydroDispatch
ProductionCostExpression__ThermalStandard
ProductionCostExpression__RenewableDispatch
ProductionCostExpression__HydroDispatch
RequirementTimeSeriesParameter__VariableReserve__ReserveDown__Reg_Down
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Reg_Up
ActivePowerTimeSeriesParameter__RenewableDispatch
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Spin_Up_R2
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Spin_Up_R1
ActivePowerTimeSeriesParameter__PowerLoad
OnStatusParameter__ThermalStandard


## Helper functions provide easy access to container contents

In [94]:
list_variable_names(uc_results)

11-element Vector{String}:
 "ActivePowerVariable__RenewableDispatch"
 "ActivePowerReserveVariable__VariableReserve__ReserveUp__Spin_Up_R2"
 "ActivePowerReserveVariable__VariableReserve__ReserveUp__Spin_Up_R1"
 "OnVariable__ThermalStandard"
 "ActivePowerReserveVariable__VariableReserve__ReserveUp__Spin_Up_R3"
 "StartVariable__ThermalStandard"
 "ActivePowerReserveVariable__VariableReserve__ReserveUp__Reg_Up"
 "ActivePowerVariable__HydroDispatch"
 "ActivePowerReserveVariable__VariableReserve__ReserveDown__Reg_Down"
 "StopVariable__ThermalStandard"
 "ActivePowerVariable__ThermalStandard"

## Results retain sequence structure

In [97]:
read_variable(uc_results, "ActivePowerVariable__ThermalStandard")[DateTime("2020-01-01T00:00:00")]

,DateTime,322_CT_6,321_CC_1,202_STEAM_3,223_CT_4,123_STEAM_2,213_CT_1,223_CT_6,313_CC_1,101_STEAM_3,123_CT_1,113_CT_3,302_CT_3,215_CT_4,102_STEAM_3,301_CT_4,113_CT_2,221_CC_1,223_CT_5,315_CT_7,215_CT_5,323_CC_1,101_STEAM_4,113_CT_1,223_STEAM_3,318_CC_1,307_CT_2,213_CT_2,113_CT_4,118_CC_1,315_CT_8,218_CC_1,213_CC_3,115_STEAM_3,323_CC_2,322_CT_5,207_CT_2,123_STEAM_3,223_STEAM_1,123_CT_5,223_STEAM_2,202_STEAM_4,116_STEAM_1,121_NUCLEAR_1,123_CT_4
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-01T00:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,202.833,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
2,2020-01-01T01:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,192.333,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,397.333,22.0
3,2020-01-01T02:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,202.833,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,397.333,22.0
4,2020-01-01T03:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,206.333,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
5,2020-01-01T04:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,211.583,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
6,2020-01-01T05:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,216.833,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
7,2020-01-01T06:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,229.083,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
8,2020-01-01T07:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,239.583,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,400.0,22.0
9,2020-01-01T08:00:00,22.0,170.0,0.0,22.0,0.0,22.0,22.0,231.667,0.0,22.0,22.0,22.0,22.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,170.0,0.0,22.0,140.0,170.0,22.0,22.0,22.0,170.0,22.0,243.083,170.0,0.0,170.0,22.0,22.0,140.0,0.0,22.0,0.0,0.0,0.0,397.333,22.0


In [96]:
ENV["COLUMNS"] = 500

500

## Realized results indicate contiguous scheduling decisions

In [98]:
x = read_realized_variable(uc_results, "ActivePowerVariable__ThermalStandard")

,DateTime,101_STEAM_3,101_STEAM_4,102_STEAM_3,102_STEAM_4,107_CC_1,113_CT_1,113_CT_2,113_CT_3,113_CT_4,115_STEAM_3,116_STEAM_1,118_CC_1,121_NUCLEAR_1,123_CT_1,123_CT_4,123_CT_5,123_STEAM_2,123_STEAM_3,201_STEAM_3,202_STEAM_3,202_STEAM_4,207_CT_1,207_CT_2,213_CC_3,213_CT_1,213_CT_2,215_CT_4,215_CT_5,216_STEAM_1,218_CC_1,221_CC_1,223_CT_4,223_CT_5,223_CT_6,223_STEAM_1,223_STEAM_2,223_STEAM_3,301_CT_3,301_CT_4,302_CT_3,302_CT_4,307_CT_1,307_CT_2
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-01T00:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,202.833,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
2,2020-01-01T01:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,397.333,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,192.333,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
3,2020-01-01T02:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,397.333,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,202.833,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
4,2020-01-01T03:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,206.333,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
5,2020-01-01T04:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,211.583,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
6,2020-01-01T05:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,216.833,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
7,2020-01-01T06:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,229.083,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
8,2020-01-01T07:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,400.0,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,239.583,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0
9,2020-01-01T08:00:00,0.0,0.0,0.0,0.0,170.0,22.0,22.0,22.0,22.0,0.0,0.0,170.0,397.333,22.0,22.0,22.0,0.0,140.0,0.0,0.0,0.0,22.0,22.0,170.0,22.0,22.0,22.0,22.0,0.0,243.083,170.0,22.0,22.0,22.0,0.0,0.0,140.0,22.0,22.0,22.0,22.0,22.0,22.0


## Plotting
The [PowerGraphics.jl](https://github.com/nrel-siip/powergraphics.jl) package has some standard plotting capabilitites based on the results produduced by PowerSimulations.jl

In [99]:
set_system!(uc_results, sys)
p = plot_fuel(uc_results);

data: [
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, plot, showlegend, stackgroup, type, x, and y"
]

layout: "layout with fields barmode, line, linestyle, linewidth, margin, nofill, seriescolor, set, stack, template, title, xaxis, y, and yaxis"

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

...

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

### Takeaways:

- Produciton Cost Modeling has a variety of usages and requires a __flexible framework__ to enable more than a single application
- SIIP enables __flexible definition of simulation sequencing__ (the hard part) and problem specifications
- The Julia ecosystem allows us to build powerful and extensible __modeling and analysis__ tools
- Ultimately, SIIP has proven __scalable and fast__ enough for applied studies, __flexible__ enough to suport technology exploration, and __extensible__ enough to support algorithmic development

 [__PowerSystems.jl__](https://github.com/nrel-siip/powersystems.jl)
 
 [__PowerSimulations.jl__](https://github.com/nrel-siip/powersimulations.jl)   
 
 [__PowerSimulationsDynamics.jl__](https://github.com/nrel-siip/powersimulationsdynamics.jl)   
 
 [__PowerGraphics.jl__](https://github.com/nrel-siip/powergraphics.jl)   
 
 [__SIIPExamples.jl__](https://github.com/nrel-siip/siipexamples.jl)   

### What's Next?
 - Performance improvements
 - Features: *hybrid plants, demand response agents, outages, contingency analysis, etc.*
 - Integrated workflows: *PCM -> ACOPF -> CA -> Stability*
 - *Experiments* (using PowerSimulations to execute scientifically rigerous analysis... and make it reproducable)


<p style="text-align: center;"> github.com/NREL-SIIP    |    <a href="https://join.slack.com/t/nrel-siip/shared_invite/zt-glam9vdu-o8A9TwZTZqqNTKHa7q3BpQ">Slack</a> | <a href="https://www.youtube.com/channel/UCTAhWBEHijW1hXZROBYFMKA">YouTube</a> </p>

<p style="text-align: center;"> clayton.barrows@nrel.gov        |         @claytonpbarrows (GitHub, gitter, slack, twitter) </p>